#### 1. Tensorflow实现最简单的Softmax Regression手写数字图片识别

In [28]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction = 0.1)
sess = tf.InteractiveSession(config = tf.ConfigProto(gpu_options = gpu_options))

x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
y = tf.nn.softmax(tf.matmul(x, W) + b)

y_ = tf.placeholder(tf.float32, [None, 10])
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
tf.global_variables_initializer().run()

mnist = input_data.read_data_sets("../data/hand_image/", one_hot=True)
for i in range(10000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    train_step.run({x: batch_xs, y_: batch_ys})
print '>> train finished'

correct_prediction = tf.equal(tf.arg_max(y, 1), tf.arg_max(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print accuracy.eval({x: mnist.test.images, y_: mnist.test.labels})

Extracting ../data/hand_image/train-images-idx3-ubyte.gz
Extracting ../data/hand_image/train-labels-idx1-ubyte.gz
Extracting ../data/hand_image/t10k-images-idx3-ubyte.gz
Extracting ../data/hand_image/t10k-labels-idx1-ubyte.gz
>> train finished
0.9238


#### 2. Tensorflow实现自编码器

首先，我们通过tf.random_uniform创建$\left( {{\rm{ - }}\sqrt {\frac{6}{{{n_{in}} + {n_{out}}}}} ,\sqrt {\frac{6}{{{n_{in}} + {n_{out}}}}} } \right)$范围内的均匀分布，而根据它的方差公式$D(x) = \frac{{{{(max - min)}^2}}}{{12}}$刚好等于$\frac{2}{{{n_{in}} + {n_{out}}}}$，因此这里实现的就是标准的均匀分布的Xaiver初始化器，其中$fan\_in$是输入节点的数量，$fan\_out$是输出节点的数量。

In [1]:
import numpy as np
import sklearn.preprocessing as prep
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data


def xavier_init(fan_in, fan_out, constant=1):
    low = -constant * np.sqrt(6.0 / (fan_in + fan_out))
    high = constant * np.sqrt(6.0 / (fan_in + fan_out))
    return tf.random_uniform((fan_in, fan_out), 
                            minval = low, maxval = high,
                            dtype = tf.float32)


class AdditiveGaussianNoiseAutocoder(object):
    def __init__(self, n_input, n_hidden, transfer_function=tf.nn.softplus,
                optimizer = tf.train.AdamOptimizer(), scale=0.1):
        # 定义网络参数
        self.n_input = n_input
        self.n_hidden = n_hidden
        self.transfer = transfer_function
        self.scale = tf.placeholder(tf.float32)
        self.training_scale = scale
        network_weights = self._initialize_weights()
        self.weights = network_weights
        # 定义网络结构
        self.x = tf.placeholder(tf.float32, [None, self.n_input])
        self.hidden = self.transfer(tf.add(
            tf.matmul(self.x + scale * tf.random_normal((n_input,)), self.weights['w1']), 
            self.weights['b1']))
        self.reconstruction = tf.add(tf.matmul(self.hidden, self.weights['w2']), self.weights['b2'])
        # 定义自编码器的损失函数
        self.cost = 0.5 * tf.reduce_sum(tf.pow(tf.subtract(self.reconstruction, self.x), 2.0))
        self.optimizer = optimizer.minimize(self.cost)
        init = tf.global_variables_initializer()
        gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction = 0.2)
        self.sess = tf.Session(config = tf.ConfigProto(gpu_options = gpu_options))
        self.sess.run(init)
    
    def _initialize_weights(self):
        all_weights = dict()
        all_weights['w1'] = tf.Variable(xavier_init(self.n_input, self.n_hidden))
        all_weights['b1'] = tf.Variable(tf.zeros([self.n_hidden], dtype=tf.float32))
        all_weights['w2'] = tf.Variable(tf.zeros([self.n_hidden, self.n_input], dtype=tf.float32))
        all_weights['b2'] = tf.Variable(tf.zeros([self.n_input], dtype=tf.float32))
        return all_weights
                                    
    def partial_fit(self, X):
        cost, opt = self.sess.run((self.cost, self.optimizer),
                                  feed_dict = {self.x: X, self.scale: self.training_scale})
        return cost
    
    def calc_total_cost(self, X):
        return self.sess.run(self.cost, 
                             feed_dict = {self.x: X, self.scale: self.training_scale})
    
    def transform(self, X):
        return self.sess.run(self.hidden, 
                             feed_dict = {self.x: X, self.scale: self.training_scale})
    
    def generate(self, hidden=None):
        if hidden is None:
            hidden = np.random.normal(size=self.weights['b1'])
        return self.sess.run(self.reconstruction, 
                             feed_dict = {self.hidden: self.hidden})
        
    def reconstruct(self, X):
        return self.sess.run(self.reconstruction, 
                             feed_dict = {self.x: X, self.scale: self.training_scale})
        
    def getWeights(self):
        return self.sess.run(self.weights['w1'])
    
    def getBiases(self):
        return self.sess.run(self.weights['b1'])


def standard_scale(X_train, X_test):
    preprocessor = prep.StandardScaler().fit(X_train)
    X_train = preprocessor.transform(X_train)
    X_test = preprocessor.transform(X_test)
    return X_train, X_test
    

def get_random_block_from_data(data, batch_size):
    start_index = np.random.randint(0, len(data)-batch_size)
    return data[start_index : (start_index + batch_size)]


mnist = input_data.read_data_sets("../data/hand_image/", one_hot=True)
X_train, X_test = standard_scale(mnist.train.images, mnist.test.images)
n_samples = int(mnist.train.num_examples)
training_epochs = 20
batch_size = 128
display_step = 1
auto_coder = AdditiveGaussianNoiseAutocoder(n_input=784,
                                           n_hidden=200,
                                           transfer_function=tf.nn.softplus,
                                           optimizer=tf.train.AdamOptimizer(learning_rate=0.001),
                                           scale=0.01)
for epoch in range(training_epochs):
    avg_cost = 0.
    total_batch = int(n_samples / batch_size)
    for i in range(total_batch):
        batch_xs = get_random_block_from_data(X_train, batch_size)
        cost = auto_coder.partial_fit(batch_xs)
        avg_cost += cost / n_samples * batch_size
        
    if epoch % display_step == 0:
        print ("Epoch:", '%04d' % (epoch + 1), 'cost=', "{:.9f}".format(avg_cost))

print "Total cost:", auto_coder.calc_total_cost(X_test)

Extracting ../data/hand_image/train-images-idx3-ubyte.gz
Extracting ../data/hand_image/train-labels-idx1-ubyte.gz
Extracting ../data/hand_image/t10k-images-idx3-ubyte.gz
Extracting ../data/hand_image/t10k-labels-idx1-ubyte.gz
('Epoch:', '0001', 'cost=', '20221.985465909')
('Epoch:', '0002', 'cost=', '12565.869640909')
('Epoch:', '0003', 'cost=', '11168.820194318')
('Epoch:', '0004', 'cost=', '9391.575080114')
('Epoch:', '0005', 'cost=', '9389.681082955')
('Epoch:', '0006', 'cost=', '9014.096810227')
('Epoch:', '0007', 'cost=', '9416.146805114')
('Epoch:', '0008', 'cost=', '9388.354382955')
('Epoch:', '0009', 'cost=', '8679.534415909')
('Epoch:', '0010', 'cost=', '9001.763286364')
('Epoch:', '0011', 'cost=', '8650.074213636')
('Epoch:', '0012', 'cost=', '7988.073542045')
('Epoch:', '0013', 'cost=', '8359.834574432')
('Epoch:', '0014', 'cost=', '7679.608361364')
('Epoch:', '0015', 'cost=', '7767.543963636')
('Epoch:', '0016', 'cost=', '7889.664449432')
('Epoch:', '0017', 'cost=', '8154.8